In [41]:
using LinearAlgebra
using ForwardDiff
using SparseArrays
const FD = ForwardDiff

ForwardDiff

In [42]:
function dynamics(x, u)
    # euler's with added nonlinearities
    J = Diagonal([1, 2, 3])
    ẋ = J \ (u[1:3] * sin(u[1]) - cross(x, J * x)) * cos(u[4])
end

dynamics (generic function with 1 method)

In [43]:
function rk4(x,u,dt)
    k1 = dt * dynamics(x,        u)
    k2 = dt * dynamics(x + k1/2, u)
    k3 = dt * dynamics(x + k2/2, u)
    k4 = dt * dynamics(x + k3,   u)
    x + (1 / 6) * (k1 + 2 * k2 + 2 * k3 + k4)
end

rk4 (generic function with 1 method)

In [44]:
struct IDX
    nx::Int64
    nu::Int64
    N::Int64
    nz::Int64
    nc::Int64
    x::Vector{UnitRange{Int64}}
    u::Vector{UnitRange{Int64}}
    c::Vector{UnitRange{Int64}}  
end
function create_idx(nx,nu,N)
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx
    idx_x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    idx_u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraints we're considering are dynamics constraints + initial condition + terminal condition 
    idx_c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N + 1)]
    nc = (N + 1) * nx # (N-1)*nx for dynamics, 2*nx for initial and terminal conditions
    
    return IDX(nx,nu,N,nz,nc,idx_x,idx_u,idx_c)
end

create_idx (generic function with 1 method)

In [56]:
function constraints(Z,idx,x0,xf,dt)
    c = zeros(eltype(Z[1]),idx.nc) # cre
    for i = 1:idx.N-1
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        xi₊ = Z[idx.x[i+1]] 
        c[idx.c[i]] = xi₊ - rk4(xi,ui,dt)
    end
    c[idx.c[idx.N]] = Z[idx.x[1]] - x0 ;
    c[idx.c[idx.N+1]] = Z[idx.x[idx.N]] - xf;
    return c;
end


constraints (generic function with 1 method)

In [57]:
function constraint_jacobian!(J,Z,idx,x0,xf,dt)
    for i = 1:idx.N-1
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        xi₊ = Z[idx.x[i+1]] 
        
        # diff the following constraint wrt xi, ui, xi₊
        # c[idx.c[i]] = xi₊ - rk4(xi,ui,dt)
        J[idx.c[i],idx.x[i]] = -FD.jacobian(_x->rk4(_x,ui,dt),xi)
        J[idx.c[i],idx.u[i]] = -FD.jacobian(_u->rk4(xi,_u,dt),ui)
        J[idx.c[i],idx.x[i+1]] = I(idx.nx)
    end
    
    # diff the initial and terminal condition constraints wrt x1 and xN respectively
    # c[idx.c[idx.N]] = Z[idx.x[1]] - x0 
    J[idx.c[idx.N],idx.x[1]] = I(idx.nx)
    
    # c[idx.c[idx.N+1]] = Z[idx.x[idx.N]] - xf
    J[idx.c[idx.N+1],idx.x[idx.N]] = I(idx.nx)
end

constraint_jacobian! (generic function with 1 method)

In [60]:
let 
    nx = 3
    nu = 4
    N = 10 
    idx = create_idx(nx,nu,N)
    
    x0 = randn(nx)
    xf = randn(nx)
    dt = 0.1 
    
    Z = randn(idx.nz);

    c = constraints(Z,idx,x0,xf,dt);
    J_fd = FD.jacobian(_Z->constraints(_Z,idx,x0,xf,dt),Z)
    
    J = spzeros(idx.nc,idx.nz)
    constraint_jacobian!(J,Z,idx,x0,xf,dt)
    
    @show norm(J-J_fd)  
    J
end

norm(J - J_fd) = 0.0


33×66 SparseMatrixCSC{Float64, Int64} with 222 stored entries:
⠿⠿⠿⢏⣢⣀⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠘⠛⠛⠛⣬⣦⣤⡤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠉⠉⠉⢱⣾⣶⣶⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠿⠿⠿⢏⣢⣀⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠛⠛⠛⣬⣦⣤⡤⡀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠉⢱⣾⣶⣶⠢⡀⠀⠀⠀
⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠿⠿⠿⠏⠢
⠈⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⡀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈